In [20]:
import numpy as np
import pandas as pd

import time
from pathlib import Path

# pd.set_option('display.max_colwidth', None) # show complete content of a cells in table
pd.set_option('display.max_columns', None)

# print multiple outputs in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# load data frame
# df = pd.read_excel(r"C:\Users\g21728\Desktop\Projekte\Glassdoor\Daten 2020\Fertige Dateien\1-1000.xlsx", sheet_name="Fertig")
# df = pd.read_excel(r"C:\Users\g21728\Desktop\Projekte\Glassdoor\Daten 2020\Fertige Dateien\1001-2000.xlsx", sheet_name="Fertig")
# df = pd.read_excel(r"C:\Users\g21728\Desktop\Projekte\Glassdoor\Daten 2020\Fertige Dateien\2001-3000.xlsx", sheet_name="Fertig")

# feather data file way faster than excel/csv
# df.to_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\1_1000.ftr")
# df.to_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\1001_2000.ftr")
# df.to_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\2001_3000.ftr")

# df1 = pd.read_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\1_1000.ftr")
# df2 = pd.read_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\1001_2000.ftr")
# df3 = pd.read_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\2001_3000.ftr")

# merge the 3 files
# frames = [df1, df2, df3]
# df = pd.concat(frames, ignore_index=True)
# df.to_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\1_3000.ftr")

# df = pd.read_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\1_3000.ftr")
df = pd.read_excel(r"W:\019_Glassdoor\Glassdoor-Project\3 Preprocessing\Test_Data_Glassdoor.xlsx")

# combine dfs

In [51]:
import os

# create an empty pandas data frame
df_raw = pd.DataFrame()

dir = r"D:\Docs\Dokumente\WMA\Projects\3 Glassdoor\1 Data\0 Outdated\2019_Glassdoor Data\Aggregated Reviews"
 
# iterate over all files within folder
for file in sorted(os.listdir(dir), key=len):
    if file.endswith(".csv"):
        df_raw = pd.concat([df_raw , pd.read_csv(os.path.join(dir, file), sep=";", decimal=",", encoding='unicode_escape')], axis=0)
    elif file.endswith(".xlsx"):
        df_raw = pd.concat([df_raw , pd.read_excel(os.path.join(dir, file), sheet_name="Fertig")], axis=0)
df_raw.reset_index(drop=True, inplace=True) # reset the index

# clean data

In [158]:
df = df_raw.copy()

# drop around 700 rows which have a wrong column allocation (part of the company name was shifted to the date col)
df = df.drop(df[df["rating_overall"].str.len() > 3].index)

# drop rows containing only empty values
df.dropna(axis=0, how='all', inplace=True) 
# drop rows with missing ratings

df = df[df['rating_overall'].notna() & df['rating_balance'].notna() & df['rating_culture'].notna() & df['rating_career'].notna() & df['rating_comp'].notna() & df['rating_mgmt'].notna()]

# convert to int columns
# df.dtypes # datatype of columns
# object to float
for col in ['rating_overall', 'rating_balance', 'rating_culture', 'rating_career', 'rating_comp', 'rating_mgmt', 'helpful']:
    df[col] = df[col].astype(float)
# float to int (decimal irrelevant)
float_columns = df.select_dtypes(include=np.number).columns.tolist()
df[float_columns] = df[float_columns].astype("Int64") #Int64 can store missing values as NaN

# employee_status -> categories (0: former; 1: current)
# df["employee_status"].unique()
employee_status_categories = {'Current Employee': 1, 'Former Employee': 0} # dict with categories
df["employee_status"] = np.where(df['employee_status'].str.contains('former', case=False) == True, "Former Employee", "Current Employee") # if recommends column contains "outlook", then copy this value to new outlook column. else take value from outlook column
df["employee_status"] = df["employee_status"].replace(employee_status_categories).astype("Int64") # replace strings with ints
df.rename(columns={"employee_status": "current_employee"}, inplace=True) #inplace works with the current dataset. otherwise df["employee_status"] = ... is needed in front

# assign outlook categories
# print(df["positive_outlook"].unique()) #['Positive Outlook' nan 'Negative Outlook' 'Neutral Outlook']
positive_outlook_categories = {'Positive Outlook':2, "Negative Outlook":0, "Neutral Outlook":1}
df["positive_outlook"] = df["positive_outlook"].replace(positive_outlook_categories).astype("Int64")

# years at company -> tenure categories & full (1) vs part time (0)
# print(df["years_at_company"].unique()) # show unique values
df.insert(6, "full_time", "")
employee_status_categories = {True: 1, False: 0}
df["full_time"] = df["years_at_company"].str.contains("full-time").map(employee_status_categories).astype("Int64") # if string contains "full-time"

# assign tenure categories
# tenure: cut "years at company" after "for"
df.insert(7, "tenure", "")
df["tenure"] = df["years_at_company"].str.split('for ').str[1]
# df["tenure"].unique()
tenure_categories = {'more than 3 years':3, 'more than a year':2, 'more than 5 years':4,'more than 8 years':5, 'less than a year':1,'more than 10 years':6}
df["tenure"] = df["tenure"].replace(tenure_categories).astype("Int64")

# correctly categorize erroneous "recommends" values
# print(df["recommends"].unique()) # -> ['Recommends' nan 'Approves of CEO' 'Positive Outlook' "Doesn't Recommend" 'Neutral Outlook' 'Negative Outlook' 'No opinion of CEO', 'Disapproves of CEO']
# df.insert(df.columns.get_loc("recommends")+1, "outlook_new", "") # insert new column after date column
df["positive_outlook"] = np.where(df['recommends'].str.contains('Outlook', case=False, regex=True) == True, df["recommends"], df["positive_outlook"]) # if recommends column contains "outlook", then copy this value to new outlook column. else take value from outlook column
# assign outlook categories
# print(df["positive_outlook"].unique()) #['Positive Outlook' nan 'Negative Outlook' 'Neutral Outlook']
positive_outlook_categories = {'Positive Outlook':2, "Negative Outlook":0, "Neutral Outlook":1}
df["positive_outlook"] = df["positive_outlook"].replace(positive_outlook_categories).astype("Int64")

# df.insert(df.columns.get_loc("recommends")+2, "approves_of_CEO_new", "") # insert new column after date column
df["approves_of_CEO"] = np.where(df['recommends'].str.contains('CEO', case=False, regex=True) == True, df["recommends"], df["approves_of_CEO"])
# assign ceo categories
# print(df["approves_of_CEO"].unique()) #['Approves of CEO' nan 'No opinion of CEO' 'Disapproves of CEO']
approves_of_CEO_categories = {'Approves of CEO':2, "Disapproves of CEO":0, "(?i)No opinion of CEO":1} #(?i) --> case insensitive 
df["approves_of_CEO"] = df["approves_of_CEO"].replace(approves_of_CEO_categories, regex=True).astype("Int64")


df["recommends"] = np.where(df['recommends'].str.contains('Outlook|CEO', case=False, regex=True) == True, np.NaN, df["recommends"]) # delete wrong values from recommend column
# assign recommends categories
# print(df["recommends"].unique()) #['Recommends' nan "Doesn't Recommend"]
recommends_categories = {'Recommends':1, "Doesn't Recommend":0}
df["recommends"] = df["recommends"].replace(recommends_categories).astype("Int64")

# split location to city & state
df.insert(df.columns.get_loc("location")+1, "state", "")
df.insert(df.columns.get_loc("location")+2, "city", "")
df["state"] = df["location"].str.split(',').str[1]
df["city"] = df["location"].str.split(',').str[0]
df.drop(columns=["location"], inplace=True)

# set date as time object
# df.iloc[1,1]
df["date"] = df["date"].str[4:15]
df["date"] = pd.to_datetime(df["date"], format="%b %d %Y")
# convert datetime to quarter
df['quarter'] = df['date'].dt.to_period('Q').dt.strftime('Q%q %Y')
df.insert(df.columns.get_loc("date")+1, "quarter", df.pop("quarter"))

# pandas NA to numpy NaN
df = df.replace(pd.NA, np.NaN)
df = df.reset_index(drop=True)

df[:3]
df.shape

(1469347, 25)

## merge identifier

In [159]:
# vlookup (innerjoin) company_name and ticker
df_ticker = pd.read_excel(r"W:\019_Glassdoor\Glassdoor-Project\3 Preprocessing\Russell3000_identifier.xlsx", sheet_name="Sheet1")
df = pd.merge(df, df_ticker, on="company_name", how="inner")

# reorder columns
cols = list(df.columns.values) # create list of all column names
first_columns = ['company_name', 'ticker', 'ISIN']
last_columns = ['full_time', 'current_employee', 'tenure', 'years_at_company', 'review_title', 'pros', 'cons', 'employee_title', 'state', 'city']
df = df[first_columns + [col for col in cols if col not in first_columns+last_columns] + last_columns]

df.head()

,company_name,ticker,ISIN,date,quarter,rating_overall,rating_balance,rating_culture,rating_career,rating_comp,rating_mgmt,recommends,positive_outlook,approves_of_CEO,helpful,advice_to_mgmt,response,full_time,current_employee,tenure,years_at_company,review_title,pros,cons,employee_title,state,city
0,AlerusFinancial,@ALRS,US01446U1034,2020-06-17,Q2 2020,5,5,5,3,3,5,1,2,2,0,NaN,NaN,1,1,3,I have been working at Alerus full-time for mo...,Great company,The care about you and your family,Lacks training new employees. Many different d...,Retirement Specialist,MN,Minneapolis
1,AlerusFinancial,@ALRS,US01446U1034,2020-06-11,Q2 2020,5,5,5,5,4,5,1,2,2,0,Keep up the good work in transparency and thou...,NaN,1,1,3,I have been working at Alerus full-time for mo...,Tremendous Company and Culture,Diversified financial services company with va...,Multiple systems that store data and informati...,Relationship Manager,MI,East Lansing
2,AlerusFinancial,@ALRS,US01446U1034,2020-06-09,Q2 2020,5,5,5,4,4,5,1,2,<NA>,0,NaN,NaN,0,0,2,I worked at Alerus part-time for more than a year,Great job for students,"Good pay for college students, flexible","Could use some newer, updates technology",Service Associate,NaN,NaN
3,AlerusFinancial,@ALRS,US01446U1034,2019-11-05,Q4 2019,3,4,3,3,2,3,<NA>,<NA>,<NA>,0,Donâ€™t pick favorites,NaN,1,1,3,I have been working at Alerus full-time for mo...,Iâ€™m not sure,"Flexible schedule, working with clients",Unable to move up,Retirement Account Analyst,MN,Albert Lea
4,AlerusFinancial,@ALRS,US01446U1034,2019-10-24,Q4 2019,4,5,5,3,5,3,1,<NA>,2,0,NaN,NaN,1,0,4,I worked at Alerus full-time for more than 5 y...,Great place to work,"Flexible, friendly enviroment, great benefits ...",Some of the managers clearly have favorites an...,Plan Administrator,NaN,NaN


## export cleaned df to feather

In [177]:
df["ticker"] = df["ticker"].astype(str)
df.to_feather(Path.joinpath(Path.home() / 'Desktop' , f'{time.strftime("%m%d%y")}_Glassdoor_1_3000.ftr'))

# groupby time

In [161]:
# generate list of numeric columns to groupby
# print(df.select_dtypes(include=np.number).columns.tolist())
numeric_columns = df.select_dtypes(include=np.number).columns.tolist()

# groupby company and quarter based on mean
gk = df.groupby(by=["company_name", "ISIN", "ticker", "quarter"], as_index=False)[numeric_columns].mean().round(4) #as_index False -> show company_name in every cell
# gk = df.groupby(by=["company_name", "ticker", "quarter"])[numeric_columns].agg(['count','mean']).round(4).reset_index()

# number of observations per firm-quarter, based on rating_overall
observations_column=df.groupby(["company_name", "quarter"])["rating_overall"].size()
# observations_column

# merge mean and observations dataframe
final = pd.merge(gk, observations_column, on=["company_name", "quarter"], how="inner", suffixes=['', '_observations'])
final.head()
final.shape

,company_name,ISIN,ticker,quarter,rating_overall,rating_balance,rating_culture,rating_career,rating_comp,rating_mgmt,recommends,positive_outlook,approves_of_CEO,helpful,full_time,current_employee,tenure,rating_overall_observations
0,1LifeHealthcare,US68269G1076,@ONEM,Q1 2014,2.8571,3.2857,3.1429,3.1429,2.2857,2.4286,0.3333,1.3333,1.0,9.1429,1.0,0.7143,2.2,7
1,1LifeHealthcare,US68269G1076,@ONEM,Q1 2016,4.25,4.5,4.5,4.0,3.0,4.25,1.0,2.0,2.0,2.75,1.0,0.75,1.75,4
2,1LifeHealthcare,US68269G1076,@ONEM,Q1 2017,3.1111,3.2222,3.6667,2.5556,3.4444,2.6667,0.625,1.5,0.875,6.5556,0.7778,0.4444,2.1429,9
3,1LifeHealthcare,US68269G1076,@ONEM,Q1 2018,3.1538,3.1538,3.5385,3.3846,2.6923,3.2308,0.5,1.3333,1.3333,6.6923,0.7692,0.6923,1.8182,13
4,1LifeHealthcare,US68269G1076,@ONEM,Q1 2019,3.2,3.9,3.6,2.8,3.6,2.9,0.5,1.3333,1.4444,4.6,0.9,0.7,2.125,10


In [168]:
final.to_csv(Path.joinpath(Path.home() / 'Desktop' , f'{time.strftime("%m%d%y")}_Glassdoor_Firm-Quarters.csv'), sep=";", decimal=",", index=False)